In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('reseau_en_arbre.csv')
df

,id_batiment,nb_maisons,infra_id,infra_type,longueur
0,E000001,4,P007111,infra_intacte,12.314461
1,E000001,4,P007983,infra_intacte,40.320929
2,E000001,4,P000308,infra_intacte,39.140799
3,E000001,4,P007819,infra_intacte,17.390464
4,E000002,1,P007111,infra_intacte,12.314461
...,...,...,...,...,...
6102,E000381,1,P008032,a_remplacer,10.520975
6103,E000381,1,P008035,a_remplacer,11.719759
6104,E000381,1,P004234,a_remplacer,13.484903
6105,E000381,1,P008036,a_remplacer,7.441760


In [3]:
df.describe()

,nb_maisons,longueur
count,6107.000000,6107.000000
mean,1.016211,29.314081
std,0.203714,21.681575
min,1.000000,0.372684
25%,1.000000,8.476727
50%,1.000000,27.771443
75%,1.000000,49.956115
max,4.000000,76.985593


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6107 entries, 0 to 6106
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_batiment  6107 non-null   object 
 1   nb_maisons   6107 non-null   int64  
 2   infra_id     6107 non-null   object 
 3   infra_type   6107 non-null   object 
 4   longueur     6107 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 238.7+ KB


In [5]:
non_existant_infra = df.loc[df['infra_type'] == 'a_remplacer']
non_existant_infra

,id_batiment,nb_maisons,infra_id,infra_type,longueur
10,E000003,1,P007113,a_remplacer,12.193509
11,E000003,1,P007984,a_remplacer,30.057598
12,E000003,1,P007823,a_remplacer,12.290283
465,E000067,1,P006193,a_remplacer,12.186388
466,E000067,1,P002851,a_remplacer,10.803202
...,...,...,...,...,...
6102,E000381,1,P008032,a_remplacer,10.520975
6103,E000381,1,P008035,a_remplacer,11.719759
6104,E000381,1,P004234,a_remplacer,13.484903
6105,E000381,1,P008036,a_remplacer,7.441760


In [55]:
diff_infra = non_existant_infra.groupby(['infra_id', 'longueur'])['nb_maisons'].agg('sum').reset_index()
diff_infra['difficule_infra'] = diff_infra['longueur'] / diff_infra['nb_maisons']

diff_infra                      

,infra_id,longueur,nb_maisons,difficule_infra
0,P000462,3.694776,3,1.231592
1,P000464,4.858753,4,1.214688
2,P000470,2.572249,6,0.428708
3,P000476,4.084845,8,0.510606
4,P000494,4.840070,19,0.254741
...,...,...,...,...
192,P008032,10.520975,7,1.502996
193,P008033,30.536827,8,3.817103
194,P008034,10.009903,1,10.009903
195,P008035,11.719759,4,2.929940


In [56]:
resultat_jointure = pd.merge(diff_infra, non_existant_infra[['infra_id','id_batiment']], how='left', on='infra_id')
resultat_jointure

,infra_id,longueur,nb_maisons,difficule_infra,id_batiment
0,P000462,3.694776,3,1.231592,E000363
1,P000462,3.694776,3,1.231592,E000370
2,P000462,3.694776,3,1.231592,E000371
3,P000464,4.858753,4,1.214688,E000358
4,P000464,4.858753,4,1.214688,E000363
...,...,...,...,...,...
572,P008035,11.719759,4,2.929940,E000377
573,P008035,11.719759,4,2.929940,E000378
574,P008035,11.719759,4,2.929940,E000380
575,P008035,11.719759,4,2.929940,E000381


In [57]:
diff_bati = resultat_jointure.groupby(['id_batiment'])['difficule_infra'].agg('sum').reset_index()
diff_bati = diff_bati.rename(columns={'difficule_infra': 'difficulte_batiment'})
diff_bati = diff_bati.sort_values(by=['difficulte_batiment'])

plan_list = []


In [64]:
while not diff_bati.empty:
   
   
    print(len(diff_bati))

    plan_list.append(diff_bati.iloc[0,0])
    
    diff_bati = diff_bati[diff_bati['id_batiment'] != diff_bati.iloc[0, 0]]

    resultat_jointure = resultat_jointure[resultat_jointure['id_batiment'] != diff_bati.iloc[0, 0]]

    print(len(resultat_jointure))


    diff_infra = non_existant_infra.groupby(['infra_id', 'longueur'])['nb_maisons'].agg('sum').reset_index()
    diff_infra['difficule_infra'] = diff_infra['longueur'] / diff_infra['nb_maisons']
    
    resultat_jointure = pd.merge(diff_infra, non_existant_infra[['infra_id','id_batiment']], how='left', on='infra_id')
    

    diff_bati = resultat_jointure.groupby(['id_batiment'])['difficule_infra'].agg('sum').reset_index()
    diff_bati = diff_bati.rename(columns={'difficule_infra': 'difficulte_batiment'})
    diff_bati = diff_bati.sort_values(by=['difficulte_batiment'])

    




     

84
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577
85
577

KeyboardInterrupt: 

In [52]:

diff_bati


,id_batiment,difficulte_batiment
38,E000228,8.775022
2,E000085,11.269978
16,E000146,11.632715
68,E000365,12.111409
39,E000229,12.202027
...,...,...
8,E000110,50.483821
22,E000177,52.746831
13,E000129,53.739723
28,E000193,53.861295


In [18]:
diff_bati.head(10)

,id_batiment,difficulte_batiment
42,E000258,1.843508
43,E000259,1.843508
35,E000225,1.843508
31,E000196,1.843508
30,E000195,1.843508
29,E000194,1.843508
67,E000364,7.118188
52,E000302,8.350195
38,E000228,8.775022
2,E000085,11.269978


In [12]:
resultat_jointure.loc[resultat_jointure['id_batiment']=='E000258'].sort_values(by='difficule_infra').drop_duplicates()

,infra_id,longueur,nb_maisons,difficule_infra,id_batiment
230,P005100,6.259757,41,0.152677,E000258
413,P008001,9.801813,40,0.245045,E000258
75,P000719,10.698159,41,0.260931,E000258
120,P000732,10.787139,41,0.263101,E000258


In [36]:
non_existant_infra.loc[df['id_batiment']=='E000258'].drop_duplicates()

,id_batiment,nb_maisons,infra_id,infra_type,longueur
3267,E000258,1,P005100,fourreau_non_existant,6.259757
3268,E000258,1,P000732,fourreau_non_existant,10.787139
3269,E000258,1,P008001,fourreau_non_existant,9.801813
3270,E000258,1,P000719,fourreau_non_existant,10.698159
